In [1]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# This function generates a 2D monochrome gradient image that increases or decreases at equal intervals in 
# the vertical or horizontal direction
def get_gradient_2d(start, stop, width, height, is_horizontal):
    if is_horizontal:
        return np.tile(np.linspace(start, stop, width), (height, 1))
    else:
        return np.tile(np.linspace(start, stop, height), (width, 1)).T
    
# This functions generates a 3D gradient image
def get_gradient_3d(width, height, start_list, stop_list, is_horizontal_list):
    result = np.zeros((height, width, len(start_list)), dtype=float)

    for i, (start, stop, is_horizontal) in enumerate(zip(start_list, stop_list, is_horizontal_list)):
        result[:, :, i] = get_gradient_2d(start, stop, width, height, is_horizontal)

    return Image.fromarray(np.uint8(result))

In [ ]:
# mediapipe libraries
mp_drawing = mp.solutions.drawing_utils

# to segment the portrait of a person, and can be used for replacing or modifying the background in an image. 
# The model outputs two categories, background at index 0 and person at index 1
mp_selfie_segmentation = mp.solutions.selfie_segmentation


BG_COLOR = (85,85,85) # DARK GRAY
background_image = cv2.imread('Background_Image.jpeg')

cap = cv2.VideoCapture(1) # video input

with mp_selfie_segmentation.SelfieSegmentation(model_selection=0) as selfie_segmentation:
    bg_image = None
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Warning! Empty camera frame.")
            continue
        
        # To improve performance, you can optionally mark the image as not writeable
        image.flags.writeable = False
        results = selfie_segmentation.process(image)
        image.flags.writeable = True
        
        # pixels greater than the threshold value are given binary value 1 denoting the object and the pixel values
        # lower are given 0 denoting the background
        condition = np.stack((results.segmentation_mask,) * 3, axis= -1) > 0.2
        
        # Virtual Background
        #bg_image = cv2.resize(background_image, (image.shape[1], image.shape[0])) 
        
        # Background Blur
        #bg_image = cv2.GaussianBlur(image,(55,55),0) 
        
        # Gradient Color
        bg_image = get_gradient_3d(1280, 720, (0, 0, 192), (255, 255, 64), (True, False, False))

        # Single Color
        if bg_image is None:
            bg_image = np.zeros(image.shape, dtype = np.uint8)
            bg_image[:] = BG_COLOR
        out_image = np.where(condition, image, bg_image)
        
        cv2.imshow('Virtual Background Creation', out_image) 
        if cv2.waitKey(5) & 0xFF ==27:
            break
            
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
